In [3]:
# Imports
import inspect
import random
import tempfile
import uuid
from functools import wraps
import collections

import coloredlogs
import dask as dd
import logging
import numpy as np
import pandas as pd
from coolname import generate_slug

# Logger
logger = logging.getLogger(__name__)
coloredlogs.install(level="DEBUG")

# StackOverflow snippet #1
########################################################################################################################
# /////|   Decorator   |////////////////////////////////////////////////////////////////////////////////////////////////
########################################################################################################################


def auto_assign_arguments(function):
    @wraps(function)
    def wrapped(self, *args, **kwargs):
        _assign_args(self, list(args), kwargs, function)
        function(self, *args, **kwargs)

    return wrapped


########################################################################################################################
# /////|   Utils   |////////////////////////////////////////////////////////////////////////////////////////////////////
########################################################################################################################


def _assign_args(instance, args, kwargs, function):
    def set_attribute(instance, parameter, default_arg):
        if not (parameter.startswith("_")):
            setattr(instance, parameter, default_arg)

    def assign_keyword_defaults(parameters, defaults):
        for parameter, default_arg in zip(
            reversed(parameters), reversed(defaults)
        ):
            set_attribute(instance, parameter, default_arg)

    def assign_positional_args(parameters, args):
        for parameter, arg in zip(parameters, args.copy()):
            set_attribute(instance, parameter, arg)
            args.remove(arg)

    def assign_keyword_args(kwargs):
        for parameter, arg in kwargs.items():
            set_attribute(instance, parameter, arg)

    def assign_keyword_only_defaults(defaults):
        return assign_keyword_args(defaults)

    def assign_variable_args(parameter, args):
        set_attribute(instance, parameter, args)

    (
        POSITIONAL_PARAMS,
        VARIABLE_PARAM,
        _,
        KEYWORD_DEFAULTS,
        _,
        KEYWORD_ONLY_DEFAULTS,
        _,
    ) = inspect.getfullargspec(function)
    POSITIONAL_PARAMS = POSITIONAL_PARAMS[1:]  # remove 'self'

    if KEYWORD_DEFAULTS:
        assign_keyword_defaults(
            parameters=POSITIONAL_PARAMS, defaults=KEYWORD_DEFAULTS
        )
    if KEYWORD_ONLY_DEFAULTS:
        assign_keyword_only_defaults(defaults=KEYWORD_ONLY_DEFAULTS)
    if args:
        assign_positional_args(
            parameters=POSITIONAL_PARAMS, args=args
        )
    if kwargs:
        assign_keyword_args(kwargs=kwargs)
    if VARIABLE_PARAM:
        assign_variable_args(parameter=VARIABLE_PARAM, args=args)


########################################################################################################################

In [7]:
def generateGenome(NumberInsertionSites, GenomicPosition=None):
    genome = list(range(1, NumberInsertionSites - 1))
    InsertionSiteColumn = [0] * len(genome)
    RecombinationRate = [0.01] * len(genome)
    insertionLocation = np.random.choice(
        len(genome), nInsertionSites, replace=False
    )
    chrLocation = np.random.choice(genomeSize, nChrom, replace=False)

    insertionCounter = 1
    for i in insertionLocation:
        InsertionSite[i] = insertionCounter
        insertionCounter += 1

    insertionCounter = 1
    for i in chrLocation:
        if InsertionSite[i] != 0:
            RecombinationRate[i + 1] = 0.5
            insertionCounter += 1
        else:
            RecombinationRate[i] = 0.5
            insertionCounter += 1

    genomeDict = {
        "Position": genome,
        "RecombinationRate": RecombinationRate,
        "InsertionSite": InsertionSite,
    }
    genome = pd.DataFrame(genomeDict)
    return genome